In [ ]:
from datasets import load_dataset
import json
import os

def load_and_convert_hotpotqa(output_filepath: str):
    """
    Hugging Face datasets에서 HotpotQA 데이터셋을 로드하고
    Qwen3-4B 파인튜닝 형식으로 변환하여 저장합니다.
    'train' 스플릿을 사용하며, 질문에 대한 답변을 구성하는
    'supporting_facts'에 해당하는 문서들만 맥락으로 포함합니다.

    Args:
        output_filepath (str): 변환된 JSON 데이터를 저장할 파일 경로
    """
    print("HotpotQA 데이터셋을 로드합니다...")
    # 'hotpotqa' 데이터셋의 'fullwiki' 설정을 사용하고 'train' 스플릿을 로드
    # datasets 라이브러리가 자동으로 다운로드 및 로드합니다.
    try:
        dataset = load_dataset("hotpot_qa", "fullwiki",trust_remote_code=True)
        print("데이터셋 로드 완료.")
        print(f"데이터셋 크기: {len(dataset)}")
    except Exception as e:
        print(f"HotpotQA 데이터셋 로드 중 오류 발생: {e}")
        print("데이터셋 이름이나 설정('fullwiki')을 확인해보세요.")
        return # 오류 발생 시 함수 종료


    converted_data = []
    skipped_count = 0

    print("데이터 변환을 시작합니다...")
    # 로드된 데이터셋 순회
    for example in dataset:
        question = example['question']         # 질문 텍스트
        answer = example['answer']             # 최종 답변 텍스트
        # supporting_facts는 답변을 찾기 위해 필요한 [문서 제목, 문장 인덱스] 쌍의 리스트
        supporting_facts = example['supporting_facts']
        # context는 모든 관련/방해 문서들의 [문서 제목, [문장들]] 리스트의 리스트
        contexts = example['context']

        # supporting_facts가 비어있는 예외적인 경우는 스킵 (HotpotQA는 보통 supporting facts가 있음)
        if not supporting_facts:
            skipped_count += 1
            continue

        # supporting_facts에 해당하는 문서들만 맥락으로 추출하기 위해 필요한 문서 제목들을 집합으로 만듭니다.
        # supporting_facts의 각 항목은 [제목, 문장 인덱스] 형태입니다.
        supporting_titles = {title for title, sent_idx in supporting_facts}

        context_text = ""
        # contexts 리스트를 순회하며 supporting_titles에 해당하는 문서의 문장들을 모읍니다.
        # contexts의 각 항목은 [문서 제목, [문장들 리스트]] 형태입니다.
        for title, sentences in contexts:
            if title in supporting_titles:
                # 문서 제목과 해당 문서의 문장들을 합쳐 맥락 텍스트에 추가합니다.
                context_text += f"[ {title} ] " # 문서 제목 추가 (구분용)
                context_text += " ".join(sentences) # 해당 문서의 모든 문장 연결
                context_text += "\n\n" # 문서 간 구분자 추가 (모델이 문서 구분을 인지하도록 돕습니다)

        # 구축된 맥락 텍스트가 비어있지 않고, 질문과 답변이 유효한 경우에만 변환
        # HotpotQA는 기본적으로 답변 가능하며 answer 필드에 값이 있습니다.
        if context_text.strip() and question and answer:
            # Qwen3-4B 모델의 입력 형식 (메시지 리스트) 생성
            messages = [
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
                {"role": "user", "content": f"맥락:\n{context_text}\n질문: {question}"}, # 다중 문서 맥락임을 감안하여 줄바꿈 추가
                {"role": "assistant", "content": answer}
            ]
            converted_data.append({"messages": messages})
        else:
            skipped_count += 1
            # print(f"경고: 맥락, 질문 또는 답변이 누락된 예제 스킵: ID {example.get('id', 'ID 없음')}")


    # 변환된 데이터를 JSON 파일로 저장
    output_dir = os.path.dirname(output_filepath)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(converted_data, f, indent=4, ensure_ascii=False)

        print(f"HotpotQA 데이터 변환 완료. 총 {len(converted_data)}개의 유효한 예제가 '{output_filepath}'에 저장되었습니다.")
        if skipped_count > 0:
             print(f"처리되지 않고 스킵된 예제 {skipped_count}개 있습니다.")
    except Exception as e:
         print(f"변환된 데이터 저장 중 오류 발생: {e}")


# --- 사용 예시 ---
# 변환된 데이터를 저장할 파일 경로
output_qwen_file_hotpotqa = 'data/hotpotqa_qwen_format.json'

# 데이터 로드 및 변환 함수 실행
try:
    load_and_convert_hotpotqa(output_qwen_file_hotpotqa)
except Exception as e:
    print(f"스크립트 실행 중 예상치 못한 오류 발생: {e}")
#참고: 실제 실행 시 위 try-except 블록의 주석을 해제하고 실행하세요.

c:\Users\jiho1\anaconda3\envs\llamaenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HotpotQA 데이터셋을 로드합니다...


KeyboardInterrupt: 

In [1]:
from datasets import load_dataset
import json
import os

def load_and_convert_nq(output_filepath: str):
    """
    Hugging Face datasets에서 Natural Questions (NQ) 데이터셋을 로드하고
    Qwen3-4B 파인튜닝 형식으로 변환하여 저장합니다.
    'open' 설정을 사용하고, 학습(train) 스플릿의 답변 가능한 예제 중
    첫 번째 짧은 답변을 사용합니다.

    Args:
        output_filepath (str): 변환된 JSON 데이터를 저장할 파일 경로
    """
    print("Natural Questions 데이터셋을 로드합니다...")
    # 'natural_questions' 데이터셋의 'open' 설정을 사용하고 'train' 스플릿을 로드
    # validation 스플릿을 사용하려면 'train' 대신 'validation'으로 변경
    dataset = load_dataset('natural_questions',  split='train')
    print("데이터셋 로드 완료.")

    converted_data = []
    skipped_count = 0

    print("데이터 변환을 시작합니다...")
    # 로드된 데이터셋 순회
    for example in dataset:
        question = example['question']['text']  # 질문 텍스트
        context = example['document']['text']   # 문서 텍스트 (클리닝된 내용)
        annotations = example['annotations']    # 답변 어노테이션 리스트

        # 답변 가능한 예제 찾기 (짧은 답변이 하나라도 있는 경우)
        # NQ 데이터는 여러 어노테이터의 답변이 있을 수 있으며,
        # short_answers는 리스트 형태입니다. 여기서는 첫 번째 어노테이션의
        # 첫 번째 short_answer를 사용하겠습니다.
        has_answer = False
        answer = None
        for annotation in annotations:
            if annotation['short_answers']: # short_answers 리스트가 비어있지 않은 경우
                # 첫 번째 short_answer 텍스트를 사용
                answer = annotation['short_answers'][0]['text']
                has_answer = True
                break # 첫 번째 유효한 답변을 찾으면 중단

        if has_answer and answer: # 답변을 찾았고, 답변 텍스트가 있는 경우만 처리
            # Qwen3-4B 모델의 입력 형식 (메시지 리스트) 생성
            messages = [
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
                {"role": "user", "content": f"맥락: {context}\n질문: {question}"},
                {"role": "assistant", "content": answer}
            ]
            converted_data.append({"messages": messages})
        else:
            skipped_count += 1
            # 답변이 없는 예제를 포함하고 싶다면 이 부분을 수정

    # 변환된 데이터를 JSON 파일로 저장
    output_dir = os.path.dirname(output_filepath)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(output_filepath, 'w', encoding='utf-8') as f:
        json.dump(converted_data, f, indent=4, ensure_ascii=False)

    print(f"NQ 데이터 변환 완료. 총 {len(converted_data)}개의 예제가 '{output_filepath}'에 저장되었습니다.")
    print(f"답변이 없거나 처리되지 않은 예제 {skipped_count}개는 스킵되었습니다.")


# --- 사용 예시 ---
# 변환된 데이터를 저장할 파일 경로
output_qwen_file_nq = r'C:\Users\jiho1\Downloads\nq_qwen_format.json"'

# 데이터 로드 및 변환 함수 실행
try:
    load_and_convert_nq(output_qwen_file_nq)
except Exception as e:
    print(f"데이터 로드 또는 변환 중 오류 발생: {e}")

# 참고: 실제 실행 시 위 try-except 블록의 주석을 해제하고 실행하세요.

c:\Users\jiho1\anaconda3\envs\llamaenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Natural Questions 데이터셋을 로드합니다...


Generating validation split: 100%|██████████| 7830/7830 [01:00<00:00, 129.38 examples/s]


데이터셋 로드 완료.
데이터 변환을 시작합니다...
데이터 로드 또는 변환 중 오류 발생: 'text'


In [1]:
from datasets import load_dataset

dataset = load_dataset('natural_questions',  split='train')

c:\Users\jiho1\anaconda3\envs\llamaenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import json
import os

def convert_squad_to_qwen_format(squad_filepath: str, output_filepath: str):
    """
    SQuAD 데이터셋 JSON 파일을 읽어 Qwen3-4B 파인튜닝 형식으로 변환합니다.

    Args:
        squad_filepath (str): SQuAD 데이터셋 JSON 파일 경로 (예: train-v2.0.json)
        output_filepath (str): 변환된 JSON 데이터를 저장할 파일 경로 (예: squad_qwen_format.json)
    """
    converted_data = []

    # SQuAD JSON 파일 로드
    with open(squad_filepath, 'r', encoding='utf-8') as f:
        squad_data = json.load(f)

    # 'data' 키 아래에 있는 각 문서/주제별 항목 순회
    for entry in squad_data['data']:
        # 각 항목의 'paragraphs' 순회
        for paragraph in entry['paragraphs']:
            context = paragraph['context'] # 현재 문단의 텍스트

            # 현재 문단의 'qas' (질문-답변 쌍) 순회
            for qa in paragraph['qas']:
                question = qa['question'] # 질문 텍스트

                # SQuAD v2.0에는 답변할 수 없는 질문이 포함될 수 있습니다.
                # 여기서는 답변이 있는 경우만 처리하거나, 답변이 없는 경우를 고려할 수 있습니다.
                # 기본적인 QA 능력 향상을 위해 우선 답변이 있는 경우만 처리해 보겠습니다.
                # (v1.1 데이터셋을 사용한다면 모든 질문에 답변이 있습니다.)
                if qa['answers']:
                    # 여러 답변이 있을 수 있지만, 보통 첫 번째 답변을 사용합니다.
                    answer = qa['answers'][0]['text']

                    # Qwen3-4B 모델의 입력 형식 (메시지 리스트) 생성
                    messages = [
                        # 시스템 메시지는 필요에 따라 추가하거나 변경할 수 있습니다.
                        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
                        # 사용자 메시지: 맥락과 질문 포함
                        {"role": "user", "content": f"맥락: {context}\n질문: {question}"},
                        # 어시스턴트 메시지: 정답 답변
                        {"role": "assistant", "content": answer}
                    ]

                    converted_data.append({"messages": messages})
                else:
                    # 답변이 없는 질문을 처리하려면 이 부분을 수정해야 합니다.
                    # 예: {"role": "assistant", "content": "답변을 찾을 수 없습니다."}
                    # 또는 해당 예제를 스킵. 기본적으로는 스킵합니다.
                    pass


    # 변환된 데이터를 JSON 파일로 저장
    output_dir = os.path.dirname(output_filepath)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(output_filepath, 'w', encoding='utf-8') as f:
        # JSON 배열 형태로 저장, ensure_ascii=False로 한글 깨짐 방지
        json.dump(converted_data, f, indent=4, ensure_ascii=False)

    print(f"SQuAD 데이터 변환 완료. 총 {len(converted_data)}개의 예제가 '{output_filepath}'에 저장되었습니다.")

# --- 사용 예시 ---
# SQuAD 학습 데이터 파일 경로 (실제 파일 경로로 수정하세요)
squad_input_file = 'train-v2.0.json' # 또는 train-v2.0.json
# 변환된 데이터를 저장할 파일 경로
output_qwen_file = r"C:\Users\jiho1\Downloads\squad_qwen_format.json"


# 데이터 변환 함수 실행
try:
    convert_squad_to_qwen_format(squad_input_file, output_qwen_file)
except FileNotFoundError:
    print(f"오류: SQuAD 파일을 찾을 수 없습니다. 경로를 확인하세요: {squad_input_file}")
except Exception as e:
    print(f"변환 중 오류 발생: {e}")

# 참고: 실제 실행 시 위 try-except 블록의 주석을 해제하고 파일 경로를 올바르게 설정하세요.

SQuAD 데이터 변환 완료. 총 86821개의 예제가 'C:\Users\jiho1\Downloads\squad_qwen_format.json'에 저장되었습니다.


In [1]:
import json
import pandas as pd

with open('KorQuAD_v1.0_train.json','r',encoding='utf-8') as f:
    data = json.load(f)
    
df = pd.DataFrame(data)

In [9]:
import json
import os

def convert_squad_to_qwen_format(squad_filepath: str, output_filepath: str):
    """
    SQuAD 데이터셋 JSON 파일을 읽어 Qwen3-4B 파인튜닝 형식으로 변환합니다.

    Args:
        squad_filepath (str): SQuAD 데이터셋 JSON 파일 경로 (예: train-v2.0.json)
        output_filepath (str): 변환된 JSON 데이터를 저장할 파일 경로 (예: squad_qwen_format.json)
    """
    converted_data = []

    # SQuAD JSON 파일 로드
    with open(squad_filepath, 'r', encoding='utf-8') as f:
        squad_data = json.load(f)

    # 'data' 키 아래에 있는 각 문서/주제별 항목 순회
    for entry in squad_data['data']:
        # 각 항목의 'paragraphs' 순회
        for paragraph in entry['paragraphs']:
            context = paragraph['context'] # 현재 문단의 텍스트

            # 현재 문단의 'qas' (질문-답변 쌍) 순회
            for qa in paragraph['qas']:
                question = qa['question'] # 질문 텍스트

                # SQuAD v2.0에는 답변할 수 없는 질문이 포함될 수 있습니다.
                # 여기서는 답변이 있는 경우만 처리하거나, 답변이 없는 경우를 고려할 수 있습니다.
                # 기본적인 QA 능력 향상을 위해 우선 답변이 있는 경우만 처리해 보겠습니다.
                # (v1.1 데이터셋을 사용한다면 모든 질문에 답변이 있습니다.)
                if qa['answers']:
                    # 여러 답변이 있을 수 있지만, 보통 첫 번째 답변을 사용합니다.
                    answer = qa['answers'][0]['text']

                    # Qwen3-4B 모델의 입력 형식 (메시지 리스트) 생성
                    messages = [
                        # 시스템 메시지는 필요에 따라 추가하거나 변경할 수 있습니다.
                        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
                        # 사용자 메시지: 맥락과 질문 포함
                        {"role": "user", "content": f"맥락: {context}\n질문: {question}"},
                        # 어시스턴트 메시지: 정답 답변
                        {"role": "assistant", "content": answer}
                    ]

                    converted_data.append({"messages": messages})
                    if len(converted_data) == 50000:
                        break
                else:
                    # 답변이 없는 질문을 처리하려면 이 부분을 수정해야 합니다.
                    # 예: {"role": "assistant", "content": "답변을 찾을 수 없습니다."}
                    # 또는 해당 예제를 스킵. 기본적으로는 스킵합니다.
                    pass
            if len(converted_data) == 50000:
                break
        if len(converted_data) == 50000:
            break


    # 변환된 데이터를 JSON 파일로 저장
    output_dir = os.path.dirname(output_filepath)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(output_filepath, 'w', encoding='utf-8') as f:
        # JSON 배열 형태로 저장, ensure_ascii=False로 한글 깨짐 방지
        json.dump(converted_data, f, indent=4, ensure_ascii=False)

    print(f"SQuAD 데이터 변환 완료. 총 {len(converted_data)}개의 예제가 '{output_filepath}'에 저장되었습니다.")

# --- 사용 예시 ---
# SQuAD 학습 데이터 파일 경로 (실제 파일 경로로 수정하세요)
squad_input_file = 'KorQuAD_v1.0_train.json' # 또는 train-v2.0.json
# 변환된 데이터를 저장할 파일 경로
output_qwen_file = r"C:\Users\jiho1\Downloads\korquad_qwen_format_re.json"


# 데이터 변환 함수 실행
try:
    convert_squad_to_qwen_format(squad_input_file, output_qwen_file)
except FileNotFoundError:
    print(f"오류: SQuAD 파일을 찾을 수 없습니다. 경로를 확인하세요: {squad_input_file}")
except Exception as e:
    print(f"변환 중 오류 발생: {e}")


SQuAD 데이터 변환 완료. 총 50000개의 예제가 'C:\Users\jiho1\Downloads\korquad_qwen_format_re.json'에 저장되었습니다.
